[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engelberger/tutorials-ai4pd-2025/blob/main/tutorial_alphafold2_i89_conformations_v3.ipynb)

# Tutorial: Understanding Conformational Selection in AlphaFold2 through Coevolution Analysis

**Duration:** 90 minutes  
**Instructor:** Felipe Engelberger  
**Date:** AI4PD Workshop 2025

---

## Learning Objectives

By the end of this tutorial, you will understand:

1. **Coevolution → Structure**: How AlphaFold2's Evoformer leverages evolutionary information to predict structure
2. **MSA → Conformation**: Why MSA presence/absence determines which conformation is predicted
3. **The i89 Case Study**: How removing coevolution signal at the calcium-binding site enables alternative conformation prediction
4. **Conformational Sampling**: Using MSA subsampling and dropout to explore conformational landscapes
5. **Recycling Dynamics**: How AlphaFold2 "changes its mind" about conformations during iterative refinement

## Scientific Background

AlphaFold2's Evoformer module processes Multiple Sequence Alignments (MSAs) to extract coevolution patterns - residues that mutate together through evolution often interact in 3D space. For the i89 protein (Guo, Kortemme et al.), this coevolution signal strongly biases predictions toward the calcium-bound state. By manipulating the MSA input, we can control which conformation AlphaFold2 predicts.

## Tutorial Overview

1. **Setup and Introduction** - Prepare environment and introduce i89 protein
2. **Coevolution Analysis** - Understand the evolutionary signal in the MSA
3. **Structure Predictions** - Compare predictions with/without MSA
4. **Conformational Sampling** - Explore subsampling and dropout strategies
5. **Recycling Analysis** - Track conformational changes through iterations
6. **Results Summary** - Synthesize findings and implications


## Section 1: Environment Setup

First, we'll set up our environment with the AF2 Utils package that provides a simple wrapper around ColabDesign.


In [ ]:
#@title Install Dependencies and Import Libraries
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

# Download required files if not present
if not os.path.exists("af2_utils.py"):
    os.system("wget -q https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/af2_utils.py")
    
if not os.path.exists("logmd_utils.py"):
    os.system("wget -q https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/logmd_utils.py")

# Import packages
import af2_utils as af2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Setup environment
af2.setup_environment(verbose=False)

# Check and install dependencies if needed
status = af2.check_installation(verbose=False)
missing = [k for k, v in status.items() if not v and k != 'environment_setup']
if missing:
    af2.install_dependencies(
        install_colabdesign='colabdesign' in missing,
        install_hhsuite='hhsuite' in missing,
        download_params='alphafold_params' in missing,
        verbose=False
    )

print(f"✓ AF2 Utils v{af2.__version__} ready")
print(f"✓ LogMD: {'available' if af2.check_logmd() else 'not available (install with: pip install logmd)'}")


## Section 2: The i89 Protein System

The i89 protein is a 96-residue designed protein that exhibits two distinct conformational states:
- **State 1**: Calcium-bound conformation (typically predicted with MSA)
- **State 2**: Alternative conformation (accessible without MSA)

This conformational switching makes i89 ideal for understanding how AlphaFold2 uses evolutionary information.


In [ ]:
#@title Define i89 Sequence and Load Reference Structures

# i89 protein sequence (96 residues)
I89_SEQUENCE = "GSHMASMEDLQAEARAFLSEEMIAEFKAAFDMFDADGGGDISYKAVGTVFRMLGINPSKEVLDYLKEKIDVDGSGTIDFEEFLVLMVYIMKQDA"

# Download reference structures if needed
if not os.path.exists("state1.pdb") or not os.path.exists("state2.pdb"):
    os.system("wget -q https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/state1.pdb")
    os.system("wget -q https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/state2.pdb")

# Load reference coordinates
state1_coords = af2.load_pdb_coords("state1.pdb")
state2_coords = af2.load_pdb_coords("state2.pdb")
ref_rmsd = af2.calculate_rmsd(state1_coords, state2_coords)

print(f"i89 protein: {len(I89_SEQUENCE)} residues")
print(f"RMSD between State 1 and State 2: {ref_rmsd:.2f} Å")
print(f"Calcium-binding loop: residues 85-95")


In [ ]:
#@title Interactive 3D Overlay: State 1 vs State 2
#@markdown Explore the structural differences between the two reference conformations interactively!

if af2.check_logmd():
    overlay_traj = af2.create_reference_overlay_trajectory(
        state1_path="state1.pdb",
        state2_path="state2.pdb",
        sequence=I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,
        verbose=False
    )
    
    if overlay_traj:
        print("\n" + "="*60)
        print("INTERACTIVE 3D VIEWER:")
        print("="*60)
        print("The viewer below shows both reference structures overlaid.")
        print("You can:")
        print("  • Rotate, zoom, and pan to explore the structures")
        print("  • Use the animation controls to toggle between State 1 and State 2")
        print("  • Observe the structural differences, especially in the calcium-binding loop")
        print(f"\nView at: {overlay_traj.url}")
        print("="*60 + "\n")
        
        # Display in notebook
        try:
            from IPython.display import display, HTML
            url = overlay_traj.url
            if "?" not in url:
                url += "?"
            else:
                url += "&"
            url += "preset=polymer-cartoon&fps=10"
            
            html = f'''
            <div style="margin: 20px 0; text-align: center;">
                <h4>Reference Structures Overlay: State 1 vs State 2</h4>
                <iframe 
                    src="{url}" 
                    width="800" 
                    height="600" 
                    frameborder="0"
                    style="border: 1px solid #ccc; border-radius: 5px;">
                </iframe>
                <p style="margin-top: 10px;">
                    <a href="{url}" target="_blank">Open in new window</a>
                </p>
            </div>
            '''
            display(HTML(html))
        except ImportError:
            print(f"IPython not available. View at: {overlay_traj.url}")
else:
    print("LogMD not available - install with: pip install logmd")
    print("You can still visualize the structures using PyMOL or other viewers:")



## Section 3: MSA Generation and Coevolution Analysis

### Understanding AlphaFold2's Inputs

AlphaFold2 takes two primary inputs for structure prediction:
1. **MSA (Multiple Sequence Alignment)**: Evolutionary information from homologous sequences
2. **Deletion Matrix**: Tracks insertions/deletions across sequences

The Evoformer module processes these inputs to extract coevolution patterns, which guide structure prediction.


In [ ]:
#@title Generate MSA for i89 (Run once and reuse throughout)

# Check if MSA already exists
if os.path.exists("i89_msa.npy") and os.path.exists("i89_del_matrix.npy"):
    print("Loading existing MSA...")
    msa_full = np.load("i89_msa.npy")
    deletion_matrix = np.load("i89_del_matrix.npy")
    print(f"✓ Loaded MSA with {len(msa_full)} sequences")
else:
    print("Generating MSA using MMseqs2 (this may take 2-3 minutes)...")
    msa_full, deletion_matrix = af2.get_msa(
        sequences=[I89_SEQUENCE],
        jobname="i89_msa",
        mode="unpaired",
        cov=50,
        id=90,
        max_msa=512,
        verbose=False
    )
    # Save for reuse
    np.save("i89_msa.npy", msa_full)
    np.save("i89_del_matrix.npy", deletion_matrix)
    print(f"✓ Generated MSA with {len(msa_full)} sequences")

print(f"MSA shape: {msa_full.shape} (sequences × positions)")
print(f"Deletion matrix shape: {deletion_matrix.shape}")


### Coevolution Analysis: The Key to Understanding Conformational Selection

Coevolution reveals which residues have evolved together to maintain protein function. The Evoformer learns similar patterns to predict which residues interact in 3D space.


In [ ]:
#@title Interactive Coevolution Analysis
#@markdown Explore residue-residue coevolution with interactive heatmap. 
#@markdown Hover over the plot to see which amino acids are coevolving!

# Parse MSA file for visualization
from colabdesign.af.contrib import predict
msa_path = "i89_msa/msa.a3m"
if os.path.exists(msa_path):
    sequences, deletion_matrix_parsed = predict.parse_a3m(msa_path)
else:
    print("MSA file not found - using MSA array directly")
    # msa_full is already numeric, convert to list of arrays for MSAData
    sequences = [msa_full[i] for i in range(len(msa_full))]

# Create MSA visualizer
vis = af2.MSACoevolutionVisualizer()

# Create MSAData object
msa_data = af2.MSAData(
    array=msa_full,
    deletion_matrix=deletion_matrix,
    sequences=sequences,
    neff=len(msa_full),
    length=len(I89_SEQUENCE),
    condition_name="i89 with MSA (MMseqs2)"
)

print("Computing coevolution matrix...")
print("This uses Direct Coupling Analysis (DCA) to identify coevolving residue pairs\n")

# Compute coevolution with caching
coev_matrix = vis.compute_coevolution(msa_data)

# Create interactive coevolution plot
fig_coev = vis.plot_heatmap(
    coev_matrix, 
    title="i89 Coevolution Matrix - Interactive",
    msa_data=msa_data
)

fig_coev.show()

print("\n" + "="*60)
print("Understanding the Coevolution Plot:")
print("="*60)
print("  - Brighter colors indicate stronger coevolution")
print("  - Hover over any position to see:")
print("    * Residue identities at positions i and j")
print("    * Coevolution score")
print("    * Overall MSA statistics")
print("  - Diagonal is zeroed out (self-coevolution)")
print("\nBiological Insight:")
print("  - Strong coevolution often indicates:")
print("    1. Structural contacts (residues close in 3D)")
print("    2. Functional coupling (allosteric networks)")
print("    3. Compensatory mutations maintaining protein stability")

# Identify calcium-binding loop region (residues ~85-95)
print("\n" + "="*60)
print("CALCIUM-BINDING LOOP ANALYSIS:")
print("="*60)
print("The calcium-binding loop (around residues 85-95) shows strong coevolution.")
print("This is the region that changes conformation between State 1 and State 2!")
print(f"\nCoevolution matrix shape: {coev_matrix.shape}")
print(f"Expected sequence length: {len(I89_SEQUENCE)}")

# Extract coevolution for calcium-binding region
matrix_size = coev_matrix.shape[0]
ca_start = 84  # 0-indexed start (residue 85)
ca_end = min(96, matrix_size)  # 0-indexed end (residue 96)
ca_region = range(ca_start, ca_end)
ca_coev_submatrix = coev_matrix[np.ix_(ca_region, ca_region)]
max_ca_coev = np.max(ca_coev_submatrix) if ca_coev_submatrix.size > 0 else 0
mean_ca_coev = np.mean(ca_coev_submatrix[np.triu_indices_from(ca_coev_submatrix, k=1)]) if ca_coev_submatrix.size > 0 else 0

print(f"  Analyzing residues {ca_start+1} to {ca_end} (0-indexed: {ca_start}:{ca_end})")
print(f"  Max coevolution in Ca-binding region: {max_ca_coev:.3f}")
print(f"  Mean coevolution in Ca-binding region: {mean_ca_coev:.3f}")

# Calculate overall mean properly excluding diagonal
upper_tri_all = np.triu_indices_from(coev_matrix, k=1)
overall_mean = np.mean(coev_matrix[upper_tri_all]) if coev_matrix.size > 0 else 0
print(f"  Overall mean coevolution: {overall_mean:.3f}")

# Find top coevolving pairs involving the calcium-binding loop
upper_tri = np.triu_indices_from(coev_matrix, k=6)
coev_values = coev_matrix[upper_tri]
top_indices = np.argsort(coev_values)[-10:]  # Top 10 pairs

print("\nTop 10 coevolving residue pairs:")
for idx in reversed(top_indices):
    i, j = upper_tri[0][idx], upper_tri[1][idx]
    score = coev_values[idx]
    # Check if either residue is in Ca-binding region
    in_ca_region = (ca_start <= i < ca_end) or (ca_start <= j < ca_end)
    marker = " [Ca-binding loop]" if in_ca_region else ""
    print(f"  Positions {i+1:3d}-{j+1:3d}: {score:.4f}{marker}")


In [ ]:
#@title Compare Coevolution: With vs Without MSA
#@markdown This demonstrates how MSA depth affects coevolution signal and ultimately structure prediction

print("Creating single-sequence MSA (no homologs) for comparison...")
print("This simulates what happens when we predict without MSA context\n")

# Create single-sequence MSA
msa_single, deletion_matrix_single = af2.create_single_sequence_msa(I89_SEQUENCE)

# Create sequences array for MSAData (single sequence)
sequences_single = [np.array(list(I89_SEQUENCE))]

# Create MSAData for single sequence
msa_data_single = af2.MSAData(
    array=msa_single,
    deletion_matrix=deletion_matrix_single,
    sequences=sequences_single,
    neff=1,
    length=len(I89_SEQUENCE),
    condition_name="i89 Single Sequence (No MSA)"
)

print("Computing coevolution for single-sequence MSA...")
coev_single = vis.compute_coevolution(msa_data_single)

print("\n" + "="*60)
print("COMPARING COEVOLUTION SIGNALS:")
print("="*60)

# Compare using the comparison function
conditions = {
    "With MSA (MMseqs2)": msa_data,
    "Without MSA (Single Sequence)": msa_data_single
}

fig_main, fig_diff = af2.compare_coevolution_conditions(
    conditions,
    show_difference=True,
    reference_condition="Without MSA (Single Sequence)"
)

print("\nShowing side-by-side comparison...")
fig_main.show()

if fig_diff is not None:
    print("\nShowing difference plot...")
    print("(Positive values = stronger coevolution WITH MSA)")
    fig_diff.show()

print("\n" + "="*60)
print("KEY INSIGHTS:")
print("="*60)
print("1. WITH MSA (left):")
print("   - Clear coevolution patterns visible")
print("   - Strong signal in calcium-binding loop")
print("   - Evolutionary information guides AF2 to State 1")
print("\n2. WITHOUT MSA (right):")
print("   - Minimal coevolution (single sequence)")
print("   - No evolutionary guidance")
print("   - AF2 defaults to alternative conformation (State 2)")
print("\n3. DIFFERENCE PLOT:")
print("   - Shows regions where MSA provides most information")
print("   - Red/blue regions indicate differential coevolution")
print("   - Calcium-binding loop shows strong difference!")
print("\nThis explains WHY we see different conformations:")
print("  - With MSA: Evolutionary pressure maintains State 1 (Ca-bound)")
print("  - Without MSA: AF2 explores alternative State 2 (Ca-free)")
print("  - MSA depth directly influences conformational outcome!")


## Section 4: Structure Predictions - Testing Our Hypothesis

Now we'll test our hypothesis by comparing AlphaFold2 predictions with and without MSA. All predictions automatically save PDB files for every recycle iteration.


In [ ]:
#@title Helper Functions for Consistent Predictions and Analysis

def run_prediction_with_analysis(sequence, msa, deletion_matrix, job_name, num_seeds=3, num_recycles=3):
    """Run predictions with multiple seeds, all 5 models, and save all PDBs."""
    
    job_folder = af2.create_job_folder(sequence, job_name)
    
    # Setup model once
    model = af2.setup_model(sequence, verbose=False)
    
    # Run predictions with all models (5 models × num_seeds × num_recycles)
    print(f"Running {num_seeds} seeds × 5 models × {num_recycles + 1} recycles...")
    all_predictions = af2.predict_with_all_models(
        model,
        msa=msa,
        deletion_matrix=deletion_matrix,
        num_seeds=num_seeds,
        num_recycles=num_recycles,
        seed_start=0,
        save_pdbs=True,
        job_folder=job_folder,
        sequence=sequence,
        models=None,  # Use all 5 default models
        verbose=True
    )
    
    # Add job metadata to all predictions
    for pred in all_predictions:
        pred['job_folder'] = job_folder
        pred['job_name'] = job_name
    
    return all_predictions, job_folder

def analyze_conformational_landscape(predictions, state1_coords, state2_coords):
    """Analyze which conformations were sampled."""
    
    rmsd_data = []
    for pred in predictions:
        ca_coords = pred['structure'][:, 1, :]
        rmsd1 = af2.calculate_rmsd(ca_coords, state1_coords)
        rmsd2 = af2.calculate_rmsd(ca_coords, state2_coords)
        rmsd_data.append({
            'seed': pred['seed'],
            'model_name': pred.get('model_name', 'model_1'),  # Include model name
            'rmsd_state1': rmsd1,
            'rmsd_state2': rmsd2,
            'plddt': pred['metrics']['plddt'] * 100,
            'closer_to': 'State 1' if rmsd1 < rmsd2 else 'State 2'
        })
    
    return rmsd_data

def plot_recycling_trajectory(predictions, state1_coords, state2_coords, title):
    """Plot how conformations change during recycling."""
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    for pred in predictions:
        seed = pred['seed']
        trajectory = pred['trajectory']
        
        # Extract RMSD values for each recycle
        recycles = []
        rmsd1_vals = []
        rmsd2_vals = []
        
        for step in trajectory:
            ca_coords = step['structure'][:, 1, :]
            recycles.append(step['recycle'])
            rmsd1_vals.append(af2.calculate_rmsd(ca_coords, state1_coords))
            rmsd2_vals.append(af2.calculate_rmsd(ca_coords, state2_coords))
        
        # Plot trajectories
        axes[0].plot(recycles, rmsd1_vals, 'o-', label=f'Seed {seed}', alpha=0.7)
        axes[1].plot(recycles, rmsd2_vals, 's-', label=f'Seed {seed}', alpha=0.7)
    
    axes[0].set_xlabel('Recycle')
    axes[0].set_ylabel('RMSD to State 1 (Å)')
    axes[0].set_title(f'{title} - Distance to State 1')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_xlabel('Recycle')
    axes[1].set_ylabel('RMSD to State 2 (Å)')
    axes[1].set_title(f'{title} - Distance to State 2')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def display_logmd_embedded(trajectory, title="Interactive 3D Structure"):
    """Display LogMD trajectory embedded in notebook cell."""
    if trajectory is None:
        print("No trajectory to display")
        return
    
    try:
        from IPython.display import display, HTML
        
        # Enhanced URL with pLDDT coloring
        url = trajectory.url
        if "?" not in url:
            url += "?"
        else:
            url += "&"
        url += "preset=polymer-cartoon&plddt&fps=10"
        
        # Create embedded iframe
        html = f'''
        <div style="margin: 20px 0; text-align: center;">
            <h4>{title}</h4>
            <iframe 
                src="{url}" 
                width="900" 
                height="650" 
                frameborder="0"
                style="border: 1px solid #ccc; border-radius: 5px;">
            </iframe>
            <p style="margin-top: 10px;">
                <a href="{url}" target="_blank">Open in new window</a>
            </p>
        </div>
        '''
        display(HTML(html))
    except ImportError:
        print(f"IPython not available. View trajectory at: {trajectory.url}")

def display_logmd_comparison(traj1, traj2, title1="With MSA", title2="Without MSA"):
    """Display two LogMD trajectories side-by-side."""
    try:
        from IPython.display import display, HTML
        
        url1 = f"{traj1.url}?preset=polymer-cartoon&plddt&fps=10"
        url2 = f"{traj2.url}?preset=polymer-cartoon&plddt&fps=10"
        
        html = f'''
        <div style="margin: 20px 0;">
            <h3 style="text-align: center;">Side-by-side Comparison</h3>
            <div style="display: flex; gap: 20px; justify-content: center;">
                <div style="flex: 1; text-align: center;">
                    <h4>{title1}</h4>
                    <iframe src="{url1}" width="600" height="500" frameborder="0"
                            style="border: 1px solid #ccc; border-radius: 5px;"></iframe>
                    <p><a href="{url1}" target="_blank">Full screen</a></p>
                </div>
                <div style="flex: 1; text-align: center;">
                    <h4>{title2}</h4>
                    <iframe src="{url2}" width="600" height="500" frameborder="0"
                            style="border: 1px solid #ccc; border-radius: 5px;"></iframe>
                    <p><a href="{url2}" target="_blank">Full screen</a></p>
                </div>
            </div>
        </div>
        '''
        display(HTML(html))
    except ImportError:
        print(f"IPython not available. View trajectories:")
        print(f"  {title1}: {traj1.url}")
        print(f"  {title2}: {traj2.url}")


**Note on Default Parameters**: 

All predictions in this tutorial use the standard AlphaFold2 workflow:
- **3 seeds** (different random initializations for diversity)
- **3 recycles** (iterative refinement iterations)  
- **5 models** (all AlphaFold2 models for ensemble prediction)

These defaults provide a good balance between computational cost and prediction quality. The workflow follows the pattern from `predict.py`:
- **Seeds**: Provide different random initializations → better conformational sampling
- **Recycles**: Iteratively refine the structure → better convergence  
- **Models**: Ensemble predictions from all 5 models → more robust results

**Students are encouraged to explore these parameters later** by modifying the `num_seeds`, `num_recycles`, and `models` arguments in the prediction functions:
- More seeds (e.g., `num_seeds=5`) → better conformational sampling
- More recycles (e.g., `num_recycles=6`) → better convergence but diminishing returns
- Specific models (e.g., `models=["model_1"]`) → targeted analysis of model-specific behavior


**Note on Default Parameters**: 

All predictions in this tutorial use:
- **3 seeds** (different random initializations for diversity)
- **3 recycles** (iterative refinement iterations)  
- **5 models** (all AlphaFold2 models for ensemble)

These defaults provide a good balance between computational cost and prediction quality. Students are encouraged to explore these parameters later by modifying the `num_seeds`, `num_recycles`, and `models` arguments in the prediction functions (e.g., more seeds for better sampling, more recycles for convergence, or specific models like `models=["model_1"]` for targeted analysis).


### 4.1 Vanilla AlphaFold2 Prediction (With Full MSA)


In [ ]:
#@title Prediction with Full MSA

print("Running predictions with FULL MSA (expecting State 1)...")
predictions_with_msa, folder_with_msa = run_prediction_with_analysis(
    I89_SEQUENCE, msa_full, deletion_matrix, "i89_with_msa", num_seeds=3, num_recycles=3
)

# Analyze results
rmsd_with_msa = analyze_conformational_landscape(predictions_with_msa, state1_coords, state2_coords)

# Summary
print(f"\n PDBs saved in: {folder_with_msa}/")
print(f"\n Total predictions: {len(predictions_with_msa)} (3 seeds × 5 models)")
print("\n Results with MSA (showing final recycle for each):")
for r in rmsd_with_msa:
    print(f"  {r['model_name']}, Seed {r['seed']}: RMSD to State1={r['rmsd_state1']:.2f}Å, State2={r['rmsd_state2']:.2f}Å → {r['closer_to']}")

mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_with_msa])
mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_with_msa])
print(f"\n  Average: State1={mean_rmsd1:.2f}Å, State2={mean_rmsd2:.2f}Å")
print(f"  ✓ Prediction closer to {'State 1' if mean_rmsd1 < mean_rmsd2 else 'State 2'} (as expected with MSA)")


### 4.2 Single Sequence Prediction (No MSA)


In [ ]:
#@title Prediction without MSA (Single Sequence)

# Create single sequence MSA
msa_single, del_single = af2.create_single_sequence_msa(I89_SEQUENCE)

print("Running predictions with SINGLE SEQUENCE (expecting State 2)...")
predictions_no_msa, folder_no_msa = run_prediction_with_analysis(
    I89_SEQUENCE, msa_single, del_single, "i89_no_msa", num_seeds=3, num_recycles=3
)

# Analyze results
rmsd_no_msa = analyze_conformational_landscape(predictions_no_msa, state1_coords, state2_coords)

# Summary
print(f"\n PDBs saved in: {folder_no_msa}/")
print(f"\n Total predictions: {len(predictions_no_msa)} (3 seeds × 5 models)")
print("\n Results without MSA (showing final recycle for each):")
for r in rmsd_no_msa:
    print(f"  {r['model_name']}, Seed {r['seed']}: RMSD to State1={r['rmsd_state1']:.2f}Å, State2={r['rmsd_state2']:.2f}Å → {r['closer_to']}")

mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_no_msa])
mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_no_msa])
print(f"\n  Average: State1={mean_rmsd1:.2f}Å, State2={mean_rmsd2:.2f}Å")
print(f"  ✓ Prediction closer to {'State 2' if mean_rmsd2 < mean_rmsd1 else 'State 1'} (as hypothesized without MSA)")


### 4.3 Conformational Landscape Comparison


In [ ]:
#@title Visualize Conformational Landscape

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# RMSD landscape plot
ax = axes[0]
for r in rmsd_with_msa:
    ax.scatter(r['rmsd_state1'], r['rmsd_state2'], s=100, c='steelblue', 
              marker='o', alpha=0.7, label='With MSA' if r['seed']==0 else '')
for r in rmsd_no_msa:
    ax.scatter(r['rmsd_state1'], r['rmsd_state2'], s=100, c='coral', 
              marker='s', alpha=0.7, label='No MSA' if r['seed']==0 else '')

# Add reference point
ax.scatter([0], [ref_rmsd], marker='*', s=500, c='red', 
          label=f'State1 vs State2 ({ref_rmsd:.1f}Å)')

# Diagonal line
max_val = max(ax.get_xlim()[1], ax.get_ylim()[1])
ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)

ax.set_xlabel('RMSD to State 1 (Å)')
ax.set_ylabel('RMSD to State 2 (Å)')
ax.set_title('Conformational Landscape')
ax.legend()
ax.grid(True, alpha=0.3)

# Bar plot comparison
ax = axes[1]
conditions = ['With MSA', 'Without MSA']
state1_means = [
    np.mean([r['rmsd_state1'] for r in rmsd_with_msa]),
    np.mean([r['rmsd_state1'] for r in rmsd_no_msa])
]
state2_means = [
    np.mean([r['rmsd_state2'] for r in rmsd_with_msa]),
    np.mean([r['rmsd_state2'] for r in rmsd_no_msa])
]

x = np.arange(len(conditions))
width = 0.35
bars1 = ax.bar(x - width/2, state1_means, width, label='RMSD to State 1', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, state2_means, width, label='RMSD to State 2', color='coral', alpha=0.8)

ax.set_ylabel('RMSD (Å)')
ax.set_title('Conformational Preference')
ax.set_xticks(x)
ax.set_xticklabels(conditions)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars1, state1_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            f'{val:.1f}', ha='center', va='bottom')
for bar, val in zip(bars2, state2_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            f'{val:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n Key observation:")
print("  • WITH MSA: Coevolution signal → State 1 (Ca-bound)")
print("  • WITHOUT MSA: No evolutionary bias → State 2 (alternative)")


## Section 5: Recycling Dynamics - How AlphaFold2 "Changes Its Mind"

During recycling, AlphaFold2 iteratively refines its prediction. Interestingly, it can switch between conformations during this process.


In [ ]:
#@title Analyze Recycling Trajectories

# Plot recycling trajectories
plot_recycling_trajectory(predictions_with_msa, state1_coords, state2_coords, "With MSA")
plot_recycling_trajectory(predictions_no_msa, state1_coords, state2_coords, "Without MSA")

print("\n Recycling Insights:")
print("  • Early recycles: Large conformational changes")
print("  • Later recycles: Fine-tuning and convergence")
print("  • Notice: Some trajectories switch conformational preference during recycling!")


In [ ]:
#@title RMSD Landscape During Recycling - Tracking Conformational Changes

# Create RMSD vs RMSD landscape plots showing how predictions evolve
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# With MSA
ax = axes[0]
for pred in predictions_with_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    
    rmsd1_vals = []
    rmsd2_vals = []
    
    for step in trajectory:
        ca_coords = step['structure'][:, 1, :]
        rmsd1_vals.append(af2.calculate_rmsd(ca_coords, state1_coords))
        rmsd2_vals.append(af2.calculate_rmsd(ca_coords, state2_coords))
    
    # Plot trajectory in RMSD space
    ax.plot(rmsd1_vals, rmsd2_vals, 'o-', label=f'Seed {seed}', alpha=0.7, linewidth=2, markersize=6)
    ax.scatter([rmsd1_vals[0]], [rmsd2_vals[0]], s=200, marker='s', 
              edgecolors='black', linewidths=2, zorder=5, label=f'Start (s{seed})' if seed == 0 else '')
    ax.scatter([rmsd1_vals[-1]], [rmsd2_vals[-1]], s=200, marker='*', 
              edgecolors='black', linewidths=2, zorder=5, label=f'End (s{seed})' if seed == 0 else '')

# Add reference point
ax.scatter([0], [ref_rmsd], marker='X', s=400, c='red', 
          label=f'State1 vs State2 ({ref_rmsd:.1f}Å)', zorder=6)
ax.plot([0, 15], [0, 15], 'k--', alpha=0.3)

ax.set_xlabel('RMSD to State 1 (Å)')
ax.set_ylabel('RMSD to State 2 (Å)')
ax.set_title('With MSA - Recycling Trajectory in RMSD Space')
ax.legend(loc='upper left', fontsize=8)
ax.grid(True, alpha=0.3)

# Without MSA
ax = axes[1]
for pred in predictions_no_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    
    rmsd1_vals = []
    rmsd2_vals = []
    
    for step in trajectory:
        ca_coords = step['structure'][:, 1, :]
        rmsd1_vals.append(af2.calculate_rmsd(ca_coords, state1_coords))
        rmsd2_vals.append(af2.calculate_rmsd(ca_coords, state2_coords))
    
    # Plot trajectory in RMSD space
    ax.plot(rmsd1_vals, rmsd2_vals, 's-', label=f'Seed {seed}', alpha=0.7, linewidth=2, markersize=6)
    ax.scatter([rmsd1_vals[0]], [rmsd2_vals[0]], s=200, marker='s', 
              edgecolors='black', linewidths=2, zorder=5, label=f'Start (s{seed})' if seed == 0 else '')
    ax.scatter([rmsd1_vals[-1]], [rmsd2_vals[-1]], s=200, marker='*', 
              edgecolors='black', linewidths=2, zorder=5, label=f'End (s{seed})' if seed == 0 else '')

# Add reference point
ax.scatter([0], [ref_rmsd], marker='X', s=400, c='red', 
          label=f'State1 vs State2 ({ref_rmsd:.1f}Å)', zorder=6)
ax.plot([0, 15], [0, 15], 'k--', alpha=0.3)

ax.set_xlabel('RMSD to State 1 (Å)')
ax.set_ylabel('RMSD to State 2 (Å)')
ax.set_title('Without MSA - Recycling Trajectory in RMSD Space')
ax.legend(loc='upper left', fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Observations:")
print("  • Lines show how each prediction evolves through recycles")
print("  • Square = starting point (recycle 0)")
print("  • Star = final point (recycle 3)")
print("  • Some trajectories cross the diagonal → conformational switch!")
print("  • This shows AlphaFold2 'changing its mind' during refinement")


In [ ]:
#@title Create LogMD Trajectories for Interactive Viewing

if af2.check_logmd():
    print("Creating LogMD trajectories for side-by-side comparison...")
    
    # Create trajectory for predictions with MSA
    traj_with_msa = af2.create_trajectory_from_ensemble(
        predictions_with_msa,
        I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,
        verbose=False
    )
    
    # Create trajectory for predictions without MSA
    traj_no_msa = af2.create_trajectory_from_ensemble(
        predictions_no_msa,
        I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,
        verbose=False
    )
    
    if traj_with_msa and traj_no_msa:
        print("\n📺 Embedded 3D Viewers:")
        print("  (Viewers are embedded below - no need to open separate tabs!)")
        
        # Display side-by-side comparison
        display_logmd_comparison(
            traj_with_msa, 
            traj_no_msa, 
            title1="With MSA (State 1-like)", 
            title2="Without MSA (State 2-like)"
        )
        
        # Also display individual viewers for easier inspection
        print("\n" + "="*60)
        print("Individual Viewers:")
        print("="*60)
        display_logmd_embedded(traj_with_msa, "Predictions with MSA - All Seeds and Recycles")
        display_logmd_embedded(traj_no_msa, "Predictions without MSA - All Seeds and Recycles")
else:
    print("LogMD not available - install with: pip install logmd")


In [ ]:
#@title Create Recycle Trajectories from Saved PDBs
#@markdown Inspect all recycle iterations interactively (showing example for model_1, seed 0)

if af2.check_logmd():
    print("Creating LogMD trajectories from saved recycle PDBs...")
    print("(Showing example: model_1, seed 0 for both conditions)")
    
    # Create recycle trajectory for predictions with MSA (model_1, seed 0)
    recycle_traj_msa = af2.create_recycle_trajectory(
        job_folder=folder_with_msa,
        sequence=I89_SEQUENCE,
        model_name="model_1",  # Use model name from PDB files
        seed=0,
        align_to_first=True,  # Align all recycles to first
        project="",  # Public upload
        verbose=False
    )
    
    # Create recycle trajectory without MSA (model_1, seed 0)
    recycle_traj_no_msa = af2.create_recycle_trajectory(
        job_folder=folder_no_msa,
        sequence=I89_SEQUENCE,
        model_name="model_1",  # Use model name from PDB files
        seed=0,
        align_to_first=True,
        project="",  # Public upload
        verbose=False
    )
    
    if recycle_traj_msa:
        print("\n📺 Recycle Evolution - With MSA:")
        display_logmd_embedded(recycle_traj_msa, "Recycling Evolution (Seed 0) - With MSA")
    
    if recycle_traj_no_msa:
        print("\n📺 Recycle Evolution - Without MSA:")
        display_logmd_embedded(recycle_traj_no_msa, "Recycling Evolution (Seed 0) - Without MSA")
    
    if recycle_traj_msa and recycle_traj_no_msa:
        print("\n📊 Side-by-side Recycle Comparison:")
        display_logmd_comparison(
            recycle_traj_msa,
            recycle_traj_no_msa,
            title1="Recycling with MSA",
            title2="Recycling without MSA"
        )
        
    print("\n💡 Use the animation controls to:")
    print("  • Step through each recycle iteration")
    print("  • See how the structure refines over time")
    print("  • Notice conformational changes during recycling")
else:
    print("LogMD not available - install with: pip install logmd")


## Section 6: Conformational Sampling Strategies

Now let's explore intermediate strategies between full MSA and single sequence to see if we can sample both conformations.


### 6.1 MSA Subsampling


In [ ]:
#@title MSA Subsampling - Reducing Evolutionary Signal

# Test different MSA depths
subsample_sizes = [1, 8, 32, 128]
subsample_results = {}

for n_seq in subsample_sizes:
    print(f"\nTesting with {n_seq} sequences...")
    
    # Subsample MSA
    if n_seq == 1:
        msa_sub = msa_single
        del_sub = del_single
    else:
        indices = np.random.choice(len(msa_full), min(n_seq, len(msa_full)), replace=False)
        indices[0] = 0  # Keep query sequence
        msa_sub = msa_full[indices]
        del_sub = deletion_matrix[indices]
    
    # Run predictions
    predictions_sub, folder_sub = run_prediction_with_analysis(
        I89_SEQUENCE, msa_sub, del_sub, f"i89_msa{n_seq}", num_seeds=3, num_recycles=3
    )
    
    # Analyze
    rmsd_sub = analyze_conformational_landscape(predictions_sub, state1_coords, state2_coords)
    subsample_results[n_seq] = {
        'predictions': predictions_sub,
        'rmsd_data': rmsd_sub,
        'folder': folder_sub
    }

# Visualize results
fig, ax = plt.subplots(figsize=(10, 6))

for n_seq in subsample_sizes:
    rmsd_data = subsample_results[n_seq]['rmsd_data']
    mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_data])
    mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_data])
    
    ax.scatter([n_seq], [mean_rmsd1], s=100, c='steelblue', marker='o', alpha=0.7)
    ax.scatter([n_seq], [mean_rmsd2], s=100, c='coral', marker='s', alpha=0.7)

ax.set_xscale('log')
ax.set_xlabel('Number of MSA Sequences')
ax.set_ylabel('Mean RMSD (Å)')
ax.set_title('Conformational Preference vs MSA Depth')
ax.axhline(y=5, color='gray', linestyle='--', alpha=0.5)
ax.legend(['RMSD to State 1', 'RMSD to State 2'])
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n MSA Subsampling Results:")
print("  • Fewer sequences → Weaker evolutionary bias")
print("  • Transition occurs around 8-32 sequences")
print("  • Single sequence consistently predicts State 2")


### 6.2 Dropout Sampling


In [ ]:
#@title Dropout Sampling - Introducing Stochasticity

print("Testing dropout with full MSA...")
job_folder_dropout = af2.create_job_folder(I89_SEQUENCE, "i89_dropout")
predictions_dropout = []

for seed in range(6):  # More seeds for dropout
    print(f"  Seed {seed}...", end=" ")
    
    model = af2.setup_model(I89_SEQUENCE, verbose=False)
    
    # Enable dropout for sampling
    result = af2.predict_structure(
        model,
        msa=msa_full,
        deletion_matrix=deletion_matrix,
        num_recycles=3,
        use_dropout=True,  # Enable dropout
        seed=seed,
        verbose=False
    )
    
    # Save PDB manually since predict_structure doesn't have save_pdbs
    pdb_path = f"{job_folder_dropout}/pdb/dropout_seed{seed}_r3.pdb"
    os.makedirs(os.path.dirname(pdb_path), exist_ok=True)
    af2.save_pdb(result['structure'], I89_SEQUENCE, pdb_path, result['plddt'])
    
    result['seed'] = seed
    predictions_dropout.append(result)
    print(f"pLDDT={result['plddt'].mean()*100:.1f}%")

# Analyze dropout results
rmsd_dropout = analyze_conformational_landscape(predictions_dropout, state1_coords, state2_coords)

# Visualize
fig, ax = plt.subplots(figsize=(8, 6))

for r in rmsd_dropout:
    ax.scatter(r['rmsd_state1'], r['rmsd_state2'], s=100, c='purple', 
              marker='D', alpha=0.6, label='Dropout' if r['seed']==0 else '')

# Add reference
ax.scatter([0], [ref_rmsd], marker='*', s=500, c='red', 
          label=f'State1 vs State2')

max_val = max(ax.get_xlim()[1], ax.get_ylim()[1])
ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)

ax.set_xlabel('RMSD to State 1 (Å)')
ax.set_ylabel('RMSD to State 2 (Å)')
ax.set_title('Dropout Sampling Results')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

closer_to_state1 = sum(1 for r in rmsd_dropout if r['closer_to'] == 'State 1')
closer_to_state2 = sum(1 for r in rmsd_dropout if r['closer_to'] == 'State 2')

print(f"\n🎲 Dropout Sampling Results:")
print(f"  • {closer_to_state1} predictions closer to State 1")
print(f"  • {closer_to_state2} predictions closer to State 2")
print(f"  • Dropout introduces variability but MSA still biases toward State 1")
print(f"\n📁 PDBs saved in: {job_folder_dropout}/")


## Section 7: Results Summary and Key Takeaways

### What We Learned

1. **Coevolution → Structure**: The Evoformer learns coevolution patterns from MSAs to predict which residues interact in 3D space.

2. **MSA Controls Conformation**: 
   - Strong coevolution signal at the Ca-binding site → State 1 (Ca-bound)
   - No MSA → No evolutionary bias → State 2 (alternative)

3. **Recycling Dynamics**: AlphaFold2 can change conformational preference during recycling iterations, as visualized in the RMSD landscape plots.

4. **Sampling Strategies**:
   - **MSA Subsampling**: Reducing MSA depth weakens evolutionary bias
   - **Dropout**: Adds stochasticity but doesn't overcome strong MSA bias
   - **Single Sequence**: Most reliable for accessing alternative conformations

### Practical Implications

- To sample alternative conformations, consider running predictions without MSA
- MSA depth can be tuned to explore conformational landscapes
- All predictions save PDB files for detailed structural analysis
- LogMD trajectories enable interactive exploration of conformational ensembles
- Recycling analysis reveals when and how AlphaFold2 switches conformations

### Files Generated

All PDB files from this tutorial are saved in job folders named by condition:
- `i89_with_msa/` - Full MSA predictions (all recycles saved)
- `i89_no_msa/` - Single sequence predictions (all recycles saved)
- `i89_msa{N}/` - Subsampled MSA predictions (all recycles saved)
- `i89_dropout/` - Dropout sampling results

Each folder contains PDBs for all seeds and recycles, enabling comprehensive analysis.
